In [35]:
# here is a mathematical expression that takes 3 inputs and produces one output
import math
def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [36]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

from math import sin, cos
def gradf(a, b, c): 
  dfa = -3 * (a**2) - 0.5*(1/a**0.5)
  dfb = 3 * cos(3*b) + 2.5 * (b**1.5)
  dfc = 1/c**2
  return [dfa, dfb, dfc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [37]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
numerical_grad = [0, 0, 0] # TODO 

def fa(x):    
  h = 0.000001
  derivative = f(x+h,1,1) - f(x,1,1)
  derivative /= h
  return derivative

def fb(x):    
  h = 0.000001
  derivative = f(1,x+h,1) - f(1,x,1)
  derivative /= h
  return derivative

def fc(x):    
  h = 0.000001
  derivative = f(1,1,x+h) - f(1,1,x)
  derivative /= h
  return derivative



numerical_grad[0] = fa(2)
numerical_grad[1] = fb(3) 
numerical_grad[2] = fc(4) 

# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984299080324


In [38]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [0, 0, 0] # TODO
# -----------
def symfa(x):    
  h = 0.000001
  derivative = f(x+h,1,1) - f(x-h,1,1)
  derivative /= 2*h
  return derivative

def symfb(x):    
  h = 0.000001
  derivative = f(1,x+h,1) - f(1,x-h,1)
  derivative /= 2*h
  return derivative

def symfc(x):    
  h = 0.000001
  derivative = f(1,1,x+h) - f(1,1,x-h)
  derivative /= 2*h
  return derivative



numerical_grad2[0] = symfa(2)
numerical_grad2[1] = symfb(3) 
numerical_grad2[2] = symfc(4) 


for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390909155
OK for dim 1: expected 10.25699027111255, yours returns 10.256990273127542
OK for dim 2: expected 0.0625, yours returns 0.06250000006424727


## section 2: support for softmax

In [130]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
  
  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO 

  def __mul__(self,other): 
    other = other if isinstance(other,Value) else Value(other) 
    out = Value(self.data*other.data,(self,other),'*')
    def _backward():
      self.grad += other.data*out.grad 
      other.grad += out.grad*self.data

    out._backward = _backward
    return out 

  def __rmul__(self,other) : 
    return self*other 

  def __radd__(self,other) : 
    return self + other
  
  def __sub__(self,other) : 
    return self - other
  
  def __neg__(self) : 
    return -1*self 

  def __pow__(self,other): 
    assert(isinstance(other,(float,int))) 
    out = Value(self.data**other,(self,),f'**{other}')
    def _backward():
      self.grad = other* self.data**(other-1) * out.grad 
    out._backward = _backward 
    return out

  def __truediv__(self,next) : 
    return self*next**-1

  def __rtruediv__(self, other): # other / self
    return other * self**-1
  
  def exp(self): 
    x = self.data
    out = Value(math.exp(x),(self,),'e') 
    def _backward():  
      self.grad += out.data * out.grad
    
    out._backward = _backward
    return out 
  
  print(log(2.0))


  def log(self): 
    x = self.data 
    out = Value(math.log(x),(self,),'log')
    def _backward(): 
      self.grad += (1/self.data) * out.grad 
    
    out._backward = _backward   
    return out
  # ------

  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

0.6931471805599453


In [132]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits) 
x = math.log(probs[3].data)
y = Value(-x)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [229]:
from numpy import float64
import torch 
import torch.nn as F

a = torch.Tensor([2.0]).double()
b = torch.Tensor([3.0]).double()
c = torch.Tensor([4.0]).double() 

a.requires_grad = True
b.requires_grad = True
c.requires_grad = True
  
out = -1.0*torch.pow(a,3.0) - torch.pow(c,-1) + torch.pow(b,2.5) + torch.sin(3*b) - torch.pow(a,0.5)    

#verification

logits = torch.Tensor([0.0,3.0,-2.0,1.0]).double() 



softmax = F.Softmax(dim = -1)
probs = softmax(logits)  
probs.requires_grad = True 
loss = -1.0 * torch.log(probs[3]) 
loss.backward()
logits 

# something is incorrect with backprop doesn't calculate gradient correctly
# print(a.grad,logits[3].grad)
# print(loss.item()) 



ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  #ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  ok = 'OK' if abs(probs[dim].item() - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {probs[dim].item()}")

OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535046
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625321
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
WRONG! for dim 3: expected -0.8864503806400986, yours returns 0.11354961935990125


In [223]:
logits , probs[3] , loss

(tensor([ 0.,  3., -2.,  1.], dtype=torch.float64),
 tensor(0.1135, dtype=torch.float64, grad_fn=<SelectBackward0>),
 tensor(2.1755, dtype=torch.float64, grad_fn=<MulBackward0>))